In [2]:
import logging.config
import math
import random
import gym
import numpy as np
import pkg_resources
from gym import spaces

In [3]:
from astropy import units as u
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.maneuver import Maneuver
from IPython.display import clear_output
from time import sleep
from prettytable import PrettyTable

In [4]:
class Poliastro_env(gym.Env):
    

    def __init__(self,r_1= [-6045, -3590, 2500]<< u.km,
                      v_1= [-3.457, 6.618, 2.533]<< u.km/u.s):
        
        # Initial orbit
        self.r_1 = r_1
        self.v_1 = v_1
        self.orbit = Orbit.from_vectors(Earth, r_1, v_1)
        #Action space 
        self.action_space = spaces.Box(low=np.array([-10, -10, -10]),
                                       high=np.array([10,10,10]),
                                       dtype=np.float16)

    def step(self,action):
        
        """If crash, end the task"""
        done=True if self.ground_check() or self.too_far() else False
        
        """Take the action and make time pass"""
        dv = action << (u.m / u.s)
        self.orbit = self.orbit.apply_maneuver(Maneuver.impulse(dv))
        self.orbit=self.orbit.propagate(20<<u.min)
        
        """Get reward """
        reward=self.get_reward()
        
        return reward, done
    
    def get_reward(self):
        
        if self.check_distance()<10:
            return 1
        return 0
            
    
    def ground_check(self): 
        
        perigee=self.orbit.a*(1-self.orbit.ecc) 
    
        if perigee <= 6371<< u.km:
            return True    
        return False
    
    def too_far(self):
        
        return True if self.orbit.a>16000<< u.km else False
    
    def perfect_rendevouz(self):
        return True if self.check_distance()<=1 else False
    
    def check_distance(self):
        
        """ Goal spaceship data"""
        r = [-6045, -3490, 2500] << u.km
        v = [-3.457, 6.618, 2.533] << u.km / u.s
        nostromo = Orbit.from_vectors(Earth, r, v)
    
        """Difference in orbits shape"""
        eccentricity_diff=abs(nostromo.ecc-self.orbit.ecc)
        semi_major_ax_diff=abs(nostromo.a-self.orbit.a)
        inclination_diff=abs(nostromo.inc-self.orbit.inc)
        

        score=round(semi_major_ax_diff.value+inclination_diff.value+eccentricity_diff.value,3)
               
        return score
        

    def reset(self):
        self.orbit=Orbit.from_vectors(Earth, self.r_1, self.v_1)
        return None

    def render(self):
        pass
    

# Env Call

In [5]:
env=Poliastro_env()

/opt/anaconda3/envs/rl/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(


In [6]:
episodes=5

for episode in range(1,episodes+1):
    
    env.reset()
    done=False
    score=0
    
    while not done:
        
        #env.render()
        action=env.action_space.sample()
        reward, done = env.step(action)
        score+=reward
        
    print("Episode:{} Score:{}".format(episode, score))


NameError: name 'TIME_STEP' is not defined